In [1]:
# opencv 와 웹캠을 이용한 영상인식
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk # pip or conda install pillow
from IPython.display import Audio  #winsound  ---> window os 
import winsound
from gtts import gTTS
import playsound   # pip install playsound == 1.2.2
import os
import cv2

In [2]:
cnn = tf.keras.models.load_model('my_cnn_for_deploy.h5')

In [3]:
class_names_en=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'] # CIFAR-10의 부류 이름(영어)
class_names_fr=['avion','voiture','oiseau','chatte','biche','chienne','grenouille','jument','navire','un camion'] # CIFAR-10의 부류 이름(프랑스어)
class_names_de=['Flugzeug','Automobil','Vogel','Katze','Hirsch','Hund','Frosch','Pferd','Schiff','LKW'] # CIFAR-10의 부류 이름(독일어)
class_id = 0
tk_img = ''

In [4]:
  # filedialog를 통해 파일을 선택   
  # fname = filedialog.askopenfilename()
  # img =  Image.open(fname)

In [5]:
# 웹캠 테스트
# video = cv2.VideoCapture(0)
# while video.isOpened():
#     success,frame = video.read()
#     if success: # 성공하면
#         cv2.imshow('Camera',frame)
#         key=cv2.waitKey(1) & 0xFF # 영상 주면서 살짝 기다려야 한다.
#         if key==27: # ESC 버튼을 누르면 꺼진다.
#             break
# video.release()
# cv2.destroyAllWindows()          

In [7]:
def process_folderImage():
    # filedialog를 통해 파일을 선택   
    fname = filedialog.askopenfilename()
    img =  Image.open(fname)
    process_image(img)

def process_camImage():
    # 비디오 영상을 캡처해서 아래 프로세스를 태운다
    video = cv2.VideoCapture(0)
    while video.isOpened():
        success,frame = video.read()
        if success:
            cv2.imshow('Camera',frame)
            key=cv2.waitKey(1) & 0xFF
            if key==27:
                break
    video.release()
    cv2.destroyAllWindows()    
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame)
    process_image(img)
    
# 사용자가 선택한 영상을 인식하고 결과를 저장
def process_image(img):    
  global class_id, tk_img     # 함수외부에서 선언한 변수를 사용하기 위해 global 예약어를 사용      


  tk_img = img.resize([128,128])
  tk_img =  ImageTk.PhotoImage(tk_img)
  canvas.create_image( (canvas.winfo_width()/2, canvas.winfo_height()/2),
                      image = tk_img, anchor='center' )

  x_test=[]
  x = np.asarray(img.resize([32,32])) / 255.0
  x_test.append(x)
  x_test = np.asarray(x_test)
  res = cnn.predict(x_test)   # 영상 인식
  class_id = np.argmax(res)
  # 레이블 텍스트
  label_en['text'] = '영어 : ' + class_names_en[class_id]
  label_fr['text'] = '프랑스어 : ' + class_names_fr[class_id]
  label_de['text'] = '독일어 : ' + class_names_de[class_id]    

In [8]:
# speak en
def tts_english():
    tts = gTTS(text=class_names_en[class_id],lang='en')
    if os.path.isfile("word.mp3") : os.remove('word.mp3')
    tts.save('word.mp3')  
    playsound.playsound('word.mp3', True)
# speak fr
def tts_french():
    tts = gTTS(text=class_names_fr[class_id],lang='fr')
    if os.path.isfile("word.mp3") : os.remove('word.mp3')
    tts.save('word.mp3')  
    playsound.playsound('word.mp3', True)
# speak de
def tts_deutsch():
    tts = gTTS(text=class_names_de[class_id],lang='de')
    if os.path.isfile("word.mp3") : os.remove('word.mp3')
    tts.save('word.mp3')  
    playsound.playsound('word.mp3', True)
def quit_window():  # 프로그램 종료
    win.destroy()    

In [9]:
win = tk.Tk()
win.title('다국어 단어 공부')
win.geometry('512x500')

# 함수와 GUI 프로그램의 버튼을 연결  -- 이벤트(액션) 핸들러(함수) 등록
process_button_folder = tk.Button(win,text = '폴더이미지',command = process_folderImage)
process_button_cam = tk.Button(win,text = '캠이미지',command = process_camImage)
quit_button = tk.Button(win,text = '종료',command = quit_window)
en_button = tk.Button(win,text = '듣기',command = tts_english)
fr_button = tk.Button(win,text = '듣기',command = tts_french)
de_button = tk.Button(win,text = '듣기',command = tts_deutsch)

canvas=tk.Canvas(win,width=253,height=256,bg='cyan',bd=4)
label_en=tk.Label(win,width=16,height=1,bg='yellow',bd=4,text='영어',anchor='w')
label_fr=tk.Label(win,width=16,height=1,bg='yellow',bd=4,text='프랑스어',anchor='w')
label_de=tk.Label(win,width=16,height=1,bg='yellow',bd=4,text='독일어',anchor='w')

# 위젯 배치
process_button_folder.grid(row=0,column=0)
process_button_cam.grid(row=1,column=0)

quit_button.grid(row=2,column=0)
canvas.grid(row=0,column=1)
label_en.grid(row=1,column=1,sticky='e')
label_fr.grid(row=2,column=1,sticky='e')
label_de.grid(row=3,column=1,sticky='e')
en_button.grid(row=1,column=2,sticky='w')
fr_button.grid(row=2,column=2,sticky='w')
de_button.grid(row=3,column=2,sticky='w')

win.mainloop() # 파이썬 gui 실행